### Imports

In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  #Skip if you have already downloaded it for first time

Using cache found in C:\Users\Pc-Klinik/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-1-11 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 


In [3]:
model

AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
           

### Webcam Detection

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Model loading from checkpoint

In [3]:
import os
import json

In [4]:
ckpt_path = os.path.join(os.getcwd(), "yolov5s.pt")
class_path = os.path.join(os.getcwd(), "classes.txt")

with open(class_path, 'r') as f:
    class_dict = json.loads(f.read()) #json convert string into python object (list, string, dict, etc)
    
class_dict


{'all': None,
 'person': 0,
 'bicycle': 1,
 'car': 2,
 'motorcycle': 3,
 'airplane': 4,
 'bus': 5,
 'train': 6,
 'truck': 7,
 'boat': 8,
 'traffic light': 9,
 'fire hydrant': 10,
 'stop sign': 11,
 'parking meter': 12,
 'bench': 13,
 'bird': 14,
 'cat': 15,
 'dog': 16,
 'horse': 17,
 'sheep': 18,
 'cow': 19,
 'elephant': 20,
 'bear': 21,
 'zebra': 22,
 'giraffe': 23,
 'backpack': 24,
 'umbrella': 25,
 'handbag': 26,
 'tie': 27,
 'suitcase': 28,
 'frisbee': 29,
 'skis': 30,
 'snowboard': 31,
 'sports ball': 32,
 'kite': 33,
 'baseball bat': 34,
 'baseball glove': 35,
 'skateboard': 36,
 'surfboard': 37,
 'tennis racket': 38,
 'bottle': 39,
 'wine glass': 40,
 'cup': 41,
 'fork': 42,
 'knife': 43,
 'spoon': 44,
 'bowl': 45,
 'banana': 46,
 'apple': 47,
 'sandwich': 48,
 'orange': 49,
 'broccoli': 50,
 'carrot': 51,
 'hot dog': 52,
 'pizza': 53,
 'donut': 54,
 'cake': 55,
 'chair': 56,
 'couch': 57,
 'potted plant': 58,
 'bed': 59,
 'dining table': 60,
 'toilet': 61,
 'tv': 62,
 'laptop':

### Loading model with checkpoint (weights/parameters/biased)

In [5]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=ckpt_path) #Loading custom checkpoint model to avoid downloading model again and again

Using cache found in C:\Users\Pc-Klinik/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-1-11 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients, 16.5 GFLOPs
Adding AutoShape... 


In [6]:
# model.classes = None # all classes
model.classes = [ class_dict["car"], class_dict["bus"], class_dict["truck"], class_dict["bicycle"], class_dict["train"], class_dict["motorcycle"], class_dict["boat"] ]
model.conf = 0.5  # Detection threshold



### Run detection on custom video/ real time

In [7]:
from tqdm import tqdm
import time

In [13]:
vid_path = os.path.join(os.getcwd(), "Traffic_highway.mp4")
sample_output = os.path.join(os.getcwd(), 'sample_output.mp4')

In [16]:
vid = cv2.VideoCapture(vid_path)
total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
vid_fps = int(vid.get(cv2.CAP_PROP_FPS))
vid_width, vid_height = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# codec = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(sample_output, codec, vid_fps, (vid_width, vid_height))

start = time.time()

for _ in tqdm(range(total_frames)): #tqdm is progress bar ()
                             
    ret, frame = vid.read()
    if not ret:
        break
       
    # Make detections 
    results = model(frame)
    output = np.squeeze(results.render())
    
#     cv2.imshow('YOLO', output)
    
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break
        
#     out.write(output)
    
end = time.time()
   
vid.release()
# out.release()
cv2.destroyAllWindows()

print(f"Total processing time in sec: {end-start}")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 601/601 [03:03<00:00,  3.27it/s]

Total processing time in sec: 183.68595099449158


In [29]:
total_frames, vid_fps, vid_height, vid_width

(4509, 29, 720, 1280)

### Saving frames from video

In [17]:
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
create_dir('Saved_Frames')

In [18]:
vid_path = os.path.join(os.getcwd(), "Traffic_highway.mp4")
vid = cv2.VideoCapture(vid_path)
total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))


In [19]:
cnt=0
for i in tqdm(range(total_frames)): #tqdm is progress bar ()
                             
    ret, frame = vid.read()
    if not ret:
        break
     
    if i%59==0:
        cv2.imwrite(os.path.join(os.getcwd(),'Saved_Frames',str(cnt)+'.jpg'),frame)
        cnt = cnt+1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 601/601 [00:01<00:00, 318.35it/s]


### Model Prediction on images

In [11]:
from os import listdir
folder_dir = os.path.join(os.getcwd(),'Saved_Frames')
for image in os.listdir(folder_dir):
    print(image)

0.jpg
1.jpg
10.jpg
2.jpg
3.jpg
4.jpg
5.jpg
6.jpg
7.jpg
8.jpg
9.jpg
desktop.ini
